In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Data Load

In [78]:
colnames = ['Time', '1','2','3','4','5','6','Bonus','1_prize','2_prize','3_prize','4_prize','5_prize']
data = pd.read_csv('lotto_data.csv',names=colnames)
data.head()

,Time,1,2,3,4,5,6,Bonus,1_prize,2_prize,3_prize,4_prize,5_prize
0,968,2,5,12,14,24,39,33,1667729683,37251694,1159633,50000,5000
1,967,1,6,13,37,38,40,9,5809776094,56133103,1564923,50000,5000
2,966,1,21,25,29,34,37,36,2411303513,80376784,1708691,50000,5000
3,965,2,13,25,28,29,36,34,3403348929,86316821,1662024,50000,5000
4,964,6,21,36,38,39,43,30,2345861063,75187855,1597781,50000,5000


In [79]:
train_data = data.iloc[0:800, 1:8]
valid_data = data.iloc[800:900, 1:8]
test_data = data.iloc[900:,1:8]
print(train_data.shape, valid_data.shape, test_data.shape)

(800, 7) (100, 7) (68, 7)


# Preprocessing

In [80]:
pre_train_data = {'data':[], 'label':[]}
for i in range(800-6):
    temp = []
    for j in range(i,i+5):
        temp.append(train_data.iloc[j])
    label = [0 for i in range(45)]
    for n in train_data.iloc[i+5]:
        label[n-1] = 1
    
    pre_train_data['data'].append(temp)
    pre_train_data['label'].append(label)
    
pre_valid_data = {'data':[], 'label':[]}
for i in range(100-6):
    temp = []
    for j in range(i,i+5):
        temp.append(valid_data.iloc[j])
    label = [0 for i in range(45)]
    for n in valid_data.iloc[i+5]:
        label[n-1] = 1
    
    pre_valid_data['data'].append(temp)
    pre_valid_data['label'].append(label)
    
pre_test_data = {'data':[], 'label':[]}
for i in range(68-6):
    temp = []
    for j in range(i,i+5):
        temp.append(test_data.iloc[j])
    label = [0 for i in range(45)]
    for n in valid_data.iloc[i+5]:
        label[n-1] = 1
    
    pre_test_data['data'].append(temp)
    pre_test_data['label'].append(label)

In [81]:
pre_test_data = pd.DataFrame(pre_test_data)
pre_valid_data = pd.DataFrame(pre_valid_data)
pre_train_data = pd.DataFrame(pre_train_data)
print(pre_train_data.shape, pre_valid_data.shape, pre_test_data.shape)

(794, 2) (94, 2) (62, 2)


In [82]:
pre_train_data.to_csv('./pre_train_data.csv', index=False)
pre_valid_data.to_csv('./pre_valid_data.csv', index=False)
pre_test_data.to_csv('./pre_test_data.csv', index=False)

# Model

In [169]:
class Predictor(nn.Module):
    def __init__(self):
        super(Predictor, self).__init__()
        
        self.Conv = nn.Sequential(
            nn.Conv2d(in_channels = 1, out_channels = 4, kernel_size = 2),
            nn.BatchNorm2d(4),
            nn.LeakyReLU(0.3),
            nn.Conv2d(in_channels = 4, out_channels = 8, kernel_size = 2),
            nn.BatchNorm2d(8),
            nn.LeakyReLU(0.3),
            nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 2),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(0.3)
        ) 
        self.flatten = nn.Sequential(
            nn.Linear(128, 45),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        out = self.Conv(x)
        out = torch.reshape(out,(-1,128))
        out = self.flatten(out)
        return out

# Train

In [170]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [171]:
model = Predictor().to(device)

In [172]:
optimizer = optim.Adam(model.parameters(), lr = 0.02)
loss_func = nn.BCEWithLogitsLoss()
epochs = 1000

In [173]:
for e in range(epochs):
    losses = 0
    
    for data in np.array(pre_train_data):
        x = np.expand_dims(np.expand_dims(np.array(data[0])/45, axis=0), axis=0)
        x = torch.tensor(x, dtype=torch.float).to(device)
        y = np.expand_dims(data[1], axis=0)
        y = torch.tensor(y, dtype=torch.float).to(device)

        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y)
        loss.backward()
        optimizer.step()
        
        losses += loss.cpu().data.numpy()
    
    val_losses = 0
    with torch.no_grad():
        for data in np.array(pre_valid_data):
            x = np.expand_dims(np.expand_dims(np.array(data[0])/45, axis=0), axis=0)
            x = torch.tensor(x, dtype=torch.float).to(device)
            y = np.expand_dims(data[1], axis=0)
            y = torch.tensor(y, dtype=torch.float).to(device)

            output = model.forward(x)
            loss = loss_func(output, y)
            val_losses += loss.cpu().data.numpy()
        
    if e % 5 == 0:
        print("ep:", e, " train_loss: ", losses / 794 , " val_loss: ", val_losses / 94) 

ep: 0  train_loss:  0.6940491893880013  val_loss:  0.6931489946994376
ep: 5  train_loss:  0.693147199580291  val_loss:  0.6931473771308331
ep: 10  train_loss:  0.6931471829900814  val_loss:  0.6931472325578649
ep: 15  train_loss:  0.6931471824645996  val_loss:  0.6931471970487149
ep: 20  train_loss:  0.6931471824645996  val_loss:  0.6931471856350594
ep: 25  train_loss:  0.6931471824645996  val_loss:  0.6931471843668755
ep: 30  train_loss:  0.6931471824645996  val_loss:  0.6931471837327835
ep: 35  train_loss:  0.6931471824645996  val_loss:  0.6931471837327835
ep: 40  train_loss:  0.6931471824645996  val_loss:  0.6931471824645996


KeyboardInterrupt: 